In [ ]:
import os
from IPython.core.display import display, HTML, Markdown, clear_output, Javascript
import ipywidgets as widgets
from IPython.display import IFrame, display

def on_change(change): # si attiva quando cambio scelta dell'elemento da selezionare
    global attach
    if change['type'] == 'change' and change['name'] == 'value':
        #print ("Stai scegliendo %s" % change['new'])
        attach = change['new']
        return attach

def on_button_update(_):
    clear_output()
    #display(Javascript('''var c = IPython.notebook.get_selected_index();
                       #IPython.notebook.execute_cells([c])'''))
    allegato()

def open_attachment(attach_name):
    path_attachments = "allegati"
    path_file=str(path_attachments + "/" + attach_name)
    ext = os.path.splitext(path_file)[-1].lower()
    #print(path_file)
    #print(ext)
    #print(ext.replace('.', ''))
    if(ext=='.pdf'):
        pdf=IFrame(path_file, width=700, height=400)
        display(pdf)
    else:
        #file = open(path_file, "rb")
        display(Markdown("![alt text](" + path_file + ")"))
        delete = widgets.Button(description='Rimuovi allegato')
        delete.on_click(on_button_delete)
        display(delete)

        #image = file.read()
        #w=widgets.Image(
        #    value=image,
        #    format=ext.replace('.', ''),
        #    width=300,
        #    height=400,
        #)
        #display(w)

def on_button_confirm(_):
    global attach
    clear_output()
    print('Allegato: ' + str(attach))
    open_attachment(attach)

def allegato():
    path_attachments = "allegati"
    global attach # allegato attuale scelto
    update = widgets.Button(description='Aggiorna') #bottone visualizzato in ogni caso
    delete = widgets.Button(description='Rimuovi allegato')
    attach_list = os.listdir(path_attachments)
    if 'ck_points' in attach_list:
        attach_list.remove('ck_points')

    if(attach_list): #si attiva se c'è almeno un elemento in lista quindi nella cartella allegati
        confirm = widgets.Button(description='Conferma')
        chosen_attach_wid = widgets.Dropdown(
            options = attach_list,
            value = attach_list[0],
            description = 'Allegati:',
            disabled = False,
        )
        attach = attach_list[0]
        chosen_attach_wid.observe(on_change)
        update.on_click(on_button_update)
        confirm.on_click(on_button_confirm)
        delete.on_click(on_button_delete)
        all_buttons = [update, chosen_attach_wid, confirm, delete]
        display(widgets.HBox(all_buttons))
    else:
        print(f"ATTENZIONE: non sono presenti file nella cartella allegati.") #Ti ricordiamo che la cartella utile per gli allegati da te prodotti per questo esercizio in modo_libero di svolgimento è la cartella @path_ex_folder@/modo_libero/allegati/")
        update.on_click(on_button_update)
        delete.on_click(on_button_delete)
        all_buttons = [update, delete]
        display(widgets.HBox(all_buttons))

def on_button_delete(_):
    #delete_above_cell()
    delete_this_cell()

def delete_above_cell():
    display(Javascript('''var c = IPython.notebook.get_selected_index();
                                IPython.notebook.get_cell(c-1).metadata.editable = true;
                                IPython.notebook.get_cell(c-1).metadata.deletable = true;
                                IPython.notebook.delete_cell([c-1]);'''))
def delete_this_cell():
    display(Javascript('''var c = IPython.notebook.get_selected_index();
                                IPython.notebook.get_cell(c).metadata.editable = true;
                                IPython.notebook.get_cell(c).metadata.deletable = true;
                                IPython.notebook.delete_cell([c]);'''))

def on_button_raw_attach(_):
    add_attach_cell_din('allegato()')

def on_button_md(_):
    add_md_cell_din()

def on_button_code(_):
    add_code_cell_din()

def on_button_raw(_):
    add_raw_cell_din()

def add_raw_cell_din():
    display_id = int(time.time()*1e9) # Hack
    display(Javascript('''var idx = IPython.notebook.get_selected_index();
                       var c = IPython.notebook.insert_cell_at_index("raw", idx);
    c.set_text('');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_raw(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.get_cell(t_index).metadata.deletable = true;
    IPython.notebook.get_cell(t_index).set_text('');;'''),display_id=display_id) # Hack
    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)
    return

def add_attach_cell_din(code = ''):
    display_id = int(time.time()*1e9) # Hack
    display(Javascript('''var c = IPython.notebook.insert_cell_above();
    c.set_text(' ''' + code + ''' ');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_code(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.execute_cells([t_index]);
    IPython.notebook.get_cell(t_index).metadata.editable = false;'''),display_id=display_id) # Hack
    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)
    return

def add_code_cell_din(code = ''):
    display_id = int(time.time()*1e9)
    display(Javascript('''var c = IPython.notebook.insert_cell_above();
    c.set_text(' ''' + code + ''' ');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_code(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.execute_cells([t_index]);
    IPython.notebook.get_cell(t_index).metadata.deletable = true;
    IPython.notebook.get_cell(t_index).set_text('');'''),display_id=display_id)
    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)
    return

def add_md_cell_din():
    display_id = int(time.time()*1e9)
    display(Javascript('''var c = IPython.notebook.insert_cell_above();
    c.set_text(' ');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_markdown(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.get_cell(t_index).metadata.deletable = true;
    IPython.notebook.get_cell(t_index).set_text('');'''),display_id=display_id) # Hack
    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)
    return

def add_cell(code='', position='below', celltype='markdown', is_execute = False):
    """Create a cell in the IPython Notebook.
    code: unicode, Code to fill the new cell with.
    celltype: unicode, Type of cells "code" or "markdown".
    position: unicode, Where to put the cell "below" or "at_bottom"
    is_execute: boolean, To decide if the cell is executed after creation
    """

    # Create a unique id based on epoch time
    display_id = int(time.time()*1e9)

    if is_execute:
        display(Javascript("""
        var basis = IPython.notebook.insert_cell_{0}("{1}");
        basis.set_text(atob("{2}"));
        basis.execute();
        """.format(position, celltype, " ")),display_id=display_id)

    else:
        display(Javascript("""
        var basis = IPython.notebook.insert_cell_{0}("{1}");
        basis.set_text(atob("{2}"));
        """.format(position, celltype, " ")),display_id=display_id)


    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)

def code_button_delete():
    button_delete = widgets.Button(description="Rimuovi l'allegato", tooltip="Seleziona la cella e clicca su Elimina")
    button_delete.on_click(on_button_delete)
    #print("Vuoi eliminare la cella selezionata?")
    display(button_delete)

def loader_main():
    button_raw_attach = widgets.Button(description="Rispondi (Allegato)", tooltip="Collega un file dalla cartella attachments")
    button_md = widgets.Button(description="Rispondi (Markdown)", abstooltip="Aggiungi una cella per scrivere del testo in Markdown")
    button_code = widgets.Button(description="Rispondi (Code)", tooltip="Aggiungi una cella per scrivere del codice in Python")
    button_raw = widgets.Button(description="Rispondi (Raw)", tooltip="Aggiungi una cella per scrivere del testo libero (SCONSIGLIATA: righe troppo lunghe potrebbero fuoriuscire nella rendition, usala solo se non riesci a controllare il Markdown)")
    button_raw_attach.on_click(on_button_raw_attach)
    button_md.on_click(on_button_md)
    button_code.on_click(on_button_code)
    button_raw.on_click(on_button_raw)

    all_buttons = [button_code, button_md, button_raw, button_raw_attach]
    display(widgets.HBox(all_buttons))


# ESERCIZIO: Sottosequenze crescenti e decrescenti #


In [ ]:
import os
from IPython.core.display import display, HTML, Markdown, clear_output, Javascript
import ipywidgets as widgets
from IPython.display import IFrame, display
import numpy as np
import copy

In [ ]:
    n_tasks = 5;
    arr_point= [-1] * n_tasks;

In [ ]:
s=[63, 18, 55, 81, 7, 9, 25, 13, 31, 47, 70, 83, 4, 32, 16, 61, 43, 20, 15, 54, 63, 99, 43, 14, 27]

In [ ]:
    def is_subseq(s, subs):
        found = 0
        pos_r = 0
        if len(subs)==0:
            return False
        else:
            while pos_r < len(s):
                if s[pos_r] == subs[found]:
                    found += 1
                    if found >= len(subs):
                        return True
                pos_r += 1
        return False

    def evaluation_format(answ, pt_green,pt_red, index_pt):
        pt_blue=0
        if pt_green!=0:
            pt_blue=pt_red-pt_green
            pt_red=0
        arr_point[index_pt]=pt_green
        file = open("points.txt", "w")
        file.write(str(arr_point))
        file.close()
        return f"{answ}. Totalizzeresti <span style='color:green'>[{pt_green} safe pt]</span>,                                         <span style='color:blue'>[{pt_blue} possible pt]</span>,                                         <span style='color:red'>[{pt_red} out of reach pt]</span>.<br>"
    
    
    # Legend of the possible sequence types:
    dictionary_of_types = {
          "SC": ("implemented", "<b>strettamente crescente</b>"),
          "ND": ("implemented", "<b>non-decrescente</b>"),
          "SD": ("implemented", "<b>strettamente decrescente</b>"),
          "NC": ("implemented", "<b>non-crescente</it>"),
           "V": ("implemented", "<b>a V</b> <it>(prima giù e poi sù)</it>"),
           "A": ("implemented", "<b>ad A</b> (prima sù e poi giù)</it>"),
          "SV": ("implemented", "<b>a V stretta</b> <it>(prima strettamente giù e poi strettamente sù)</it>"),
          "SA": ("implemented", "<b>ad A stretta</b> <it>(prima strettamente sù e poi strettamente giù)</it>"),
           "N": ("implemented", "<b>a N</b> (non-decrescente con al più un ripensamento)</it>"),
           "Z": ("implemented", "<b>a Z</b> <it>(non-crescente con al più un ripensamento)</it>"),
          "SN": ("implemented", "<b>a N stretta</b> <it>(strettamente crescente con al più un ripensamento)</it>"),
          "SZ": ("implemented", "<b>a Z stretta</b> <it>(strettamente decrescente con al più un ripensamento)</it>"),
      "ZigZag": ("implemented", "<b>a Zig-Zag</b> <it>(primo passo a crescere e poi alterna ad ogni passo)</it>"),
      "ZagZig": ("implemented", "<b>a Zag-Zig</b> <it>(primo passo a calare e poi alterna ad ogni passo)</it>"),
    "ZigZagEQ": ("implemented", "<b>a Zig-Zag debole</b> <it>(primo passo a crescere e poi alterna ad ogni passo, con valori consecutivi che possono essere uguali)</it>"),
    "ZagZigEQ": ("implemented", "<b>a Zag-Zig debole</b> <it>(primo passo a calare e poi alterna ad ogni passo, con valori consecutivi che possono essere uguali)</it>"),
    "132-free": ("not yet done", "<b>dal mondo delle permutazioni pattern free per un infinità di problemi in FPT</b>"),
         "...": ("not thought of yet", "<b>???</b>")
    }
    
    def Latex_type(seq_type):
        return dictionary_of_types[seq_type][1].replace("_", "\_")
    
    
    def is_seq_of_type(s, name_s, seq_type):
        first_down = first_up = first_flat = None
        for i in range(1,len(s)):
            if s[i] < s[i-1]:
                if seq_type=="V" and first_up != None:
                    return (0,f"La sequenza ${LaTexVarName(name_s)}$ non è di tipo {Latex_type('V')} poichè ${LaTexVarName(name_s)}[${i-1}$] = {s[i-2]} $<$ {s[i-1]} $= {LaTexVarName(name_s)}[${i}$] > {LaTexVarName(name_s)}[${i+1}$] =$ {s[i]}.")
                if seq_type in {"SC","ND"} or (seq_type in {"ZigZag","ZigZagEQ"} and s[i]%2 == 1) or (seq_type in {"ZagZig","ZagZigEQ"} and s[i]%2 == 0):
                    return (0,f"La sequenza ${LaTexVarName(name_s)}$ non è di tipo {Latex_type(seq_type)} poichè ${LaTexVarName(name_s)}[${i}$] $= {s[i-1]} $>$ {s[i]} $= {LaTexVarName(name_s)}[${i+1}$]$.")
                if first_down == None:
                    first_down = i
                elif seq_type=="N":
                    return (0,f"La sequenza ${LaTexVarName(name_s)}$ non è di tipo {Latex_type(seq_type)} poichè ${LaTexVarName(name_s)}[${first_down}$] = {s[first_down-1]} $>$ {s[first_down]} $= {LaTexVarName(name_s)}[${first_down+1}$]$ e ${LaTexVarName(name_s)}[${i}$] =$ {s[i-1]} $>$ {s[i]} $= {LaTexVarName(name_s)}[${i+1}$]$.")
                if seq_type=="SN" and first_flat != None:
                    return (0,f"La sequenza ${LaTexVarName(name_s)}$ non è di tipo {Latex_type(seq_type)} poichè ${LaTexVarName(name_s)}[${first_flat}$] = {s[first_flat-1]} $=$ {s[first_flat]} $= {LaTexVarName(name_s)}[${first_flat+1}$]$ e ${LaTexVarName(name_s)}[${i}$] =$ {s[i-1]} $>$ {s[i]} $= {LaTexVarName(name_s)}[${i+1}$]$.")                
            if s[i] > s[i-1]:
                if seq_type=="A" and first_down != None:
                    return (0,f"La sequenza ${LaTexVarName(name_s)}$ non è di tipo {Latex_type('A')} poichè ${LaTexVarName(name_s)}[${i-1}$] =$ {s[i-2]} $>$ {s[i-1]} $= {LaTexVarName(name_s)}[${i}$] < {LaTexVarName(name_s)}[${i+1}$] =$ {s[i]}.")
                if seq_type in {"SD","NC"} or (seq_type in {"ZagZig","ZagZigEQ"} and s[i]%2 == 1) or (seq_type in {"ZigZag","ZigZagEQ"} and s[i]%2 == 0):
                    return (0,f"La sequenza ${LaTexVarName(name_s)}$ non è di tipo {Latex_type(seq_type)} poichè ${LaTexVarName(name_s)}[${i}$] =$ {s[i-1]} $<$ {s[i]} $= {LaTexVarName(name_s)}[${i+1}$]$.")
                if first_up == None:
                    first_up = i
                elif seq_type=="Z":
                    return (0,f"La sequenza ${LaTexVarName(name_s)}$ non è di tipo {Latex_type(seq_type)} poichè ${LaTexVarName(name_s)}[${first_up}$] =$ {s[first_up-1]} $<$ {s[first_up]} $= {LaTexVarName(name_s)}[${first_up+1}$]$ e ${LaTexVarName(name_s)}[${i}$] =$ {s[i-1]} $<$ {s[i]} $= {LaTexVarName(name_s)}[${i+1}$]$.")
                if seq_type=="SZ" and first_flat != None:
                    return (0,f"La sequenza ${LaTexVarName(name_s)}$ non è di tipo {Latex_type(seq_type)} poichè ${LaTexVarName(name_s)}[${first_flat}$] =$ {s[first_flat-1]} $=$ {s[first_flat]} $= {LaTexVarName(name_s)}[${first_flat+1}$]$ e ${LaTexVarName(name_s)}[${i}$] =$ {s[i-1]} $<$ {s[i]} $= {LaTexVarName(name_s)}[${i+1}$]$.")
            if s[i] == s[i-1]:
                if seq_type in {"SC","SD","SV","SA","ZigZag","ZagZig"}:
                    return (0,f"La sequenza ${LaTexVarName(name_s)}$ non è di tipo {Latex_type(seq_type)} poichè ${LaTexVarName(name_s)}[${i}$] =$ {s[i-1]} $=$ {s[i]} $= {LaTexVarName(name_s)}[${i+1}$]$.")
                if first_flat == None:
                    first_flat = i
                elif seq_type in {"SN","SZ"}:
                    return (0,f"La sequenza ${LaTexVarName(name_s)}$ non è di tipo {Latex_type(seq_type)} poichè ${LaTexVarName(name_s)}[${first_flat}$] =$ {s[first_flat-1]} $=$ {s[first_flat]} $= {LaTexVarName(name_s)}[${first_flat+1}$]$ e ${LaTexVarName(name_s)}[${i}$] =$ {s[i-1]} $=$ {s[i]} $= {LaTexVarName(name_s)}[${i+1}$]$.")
                if seq_type=="SN" and first_down != None:
                    return (0,f"La sequenza ${LaTexVarName(name_s)}$ non è di tipo {Latex_type(seq_type)} poichè ${LaTexVarName(name_s)}[${first_down}$] =$ {s[first_down-1]} $>$ {s[first_down]} $= {LaTexVarName(name_s)}[${first_down+1}$]$ e ${LaTexVarName(name_s)}[${i}$] =$ {s[i-1]} $=$ {s[i]} $= {LaTexVarName(name_s)}[${i+1}$]$.")
                if seq_type=="SZ" and first_up != None:
                    return (0,f"La sequenza ${LaTexVarName(name_s)}$ non è di tipo {Latex_type(seq_type)} poichè ${LaTexVarName(name_s)}[${first_up}$] =$ {s[first_up-1]} $<$ {s[first_up]} $= {LaTexVarName(name_s)}[${first_up+1}$]$ e ${LaTexVarName(name_s)}[${i}$] =$ {s[i-1]} $=$ {s[i]} $= {LaTexVarName(name_s)}[${i+1}$]$.")
        return (1,None)
    
    def LaTexVarName(var_name):
        return var_name.replace("_", "\_")
    
    
    def is_subseq_of_type(s, name_s, subs, name_subs, subs_type, pt_green, pt_red, index_pt, forced_ele_pos = None, start_banned_interval = None, end_banned_interval = None):
        submission_string = f"Hai inserito il certificato ${LaTexVarName(name_subs)}={subs}$."
        submission_string += f"<br>L'istanza era data da ${LaTexVarName(name_s)}={s}$.<br>"
    
        if not is_seq_of_type(subs, "subs", subs_type)[0]:
            return submission_string + evaluation_format("No", 0,pt_red, index_pt) + is_seq_of_type(subs, "subs", subs_type)[1]
        if start_banned_interval != None or end_banned_interval != None:
            assert start_banned_interval != None and end_banned_interval != None
            if forced_ele_pos != None:
                assert forced_ele_pos < start_banned_interval or forced_ele_pos > end_banned_interval
                if forced_ele_pos > end_banned_interval:
                    forced_ele_pos -= end_banned_interval 
            aux = s[:start_banned_interval-1] +s[end_banned_interval:]
        if not is_subseq(s, subs):
            return submission_string + f"{evaluation_format('No', 0,pt_red,index_pt)}" + f"La sequenza ${LaTexVarName(name_subs)}$ proposta non è sottosequenza di ${LaTexVarName(name_s)}$."
        if forced_ele_pos != None:
            forced_ele_0basedpos = forced_ele_pos-1
            found_magic_point = False
            for guess_0basedpos_in_subs in range(len(subs)):
                if subs[guess_0basedpos_in_subs] == s[forced_ele_0basedpos]:
                    if is_subseq(s[:forced_ele_0basedpos], subs[:guess_0basedpos_in_subs]) and is_subseq(s[forced_ele_0basedpos:], subs[guess_0basedpos_in_subs:]):
                        found_magic_point = True#False
            if not found_magic_point:
                return submission_string + f"{evaluation_format('No', 0,pt_red,index_pt)}" + f"La sequenza ${LaTexVarName(name_subs)}$ proposta non è sottosequenza di ${LaTexVarName(name_s)}$ che ne includa l'elemento in posizione ${forced_ele_pos}$."
    
        return submission_string + f"{evaluation_format('Ammissibile', pt_green,pt_red, index_pt)}"
    
    def eval_coloring(s, name_s, col, name_col, subs_type, pt_green, pt_red, index_pt):
        submission_string = f"Hai inserito il certificato ${LaTexVarName(name_col)}={col}$."
        submission_string += f"<br>L'istanza era data da ${LaTexVarName(name_s)}={s}$.<br>"
        if len(col)!=len(s):
            if len(col)>len(s):
                return f"{evaluation_format('No', 0,pt_red,index_pt)}"+f"La sequenza da te data è più lunga dell'istanza di {len(col)-len(s)}, deve essere lunga come quella di input."
            if len(col)<len(s):
                return f"{evaluation_format('No', 0,pt_red,index_pt)}"+f"La sequenza da te data deve essere lunga come quella di input: mancano {len(s)-len(col)} elementi."
        else:
            for c in col:
                subs = [s[i] for i in range(len(s)) if col[i] == c]
                if not is_seq_of_type(subs, "subs", subs_type)[0]:
                    return submission_string + f"{evaluation_format('No', 0,pt_red,index_pt)}" + f"Controllando la sottosequenza degli elementi colorati con il colore: {c} in ${LaTexVarName(name_s)}$, cioè: {subs} ... " + is_seq_of_type(subs, "subs", subs_type)[1]        
        return submission_string + f"{evaluation_format('Ammissibile', pt_green ,pt_red, index_pt)}"
    
    def min_subs_of_type(s, name_s, subs, name_subs, subs_type, pt_green, pt_red, index_pt):
        submission_string = f"Hai inserito il certificato ${LaTexVarName(name_subs)}={subs}$."
        submission_string += f"<br>L'istanza era data da ${LaTexVarName(name_s)}={s}$.<br>"
    
        check={}
        for n in s:
            if n not in check.keys():
                check[n]=1
            else:
                check[n]=check[n]+1
        for elem in subs:
            for n in elem:
                if n in check.keys():
                    check[n]=check[n]-1
        for key in check.keys():
            if check[key] != 0:
                return submission_string + f"{evaluation_format('No', 0,pt_red,index_pt)}" + f"Le tue sottosequenze non contengono tutti i valori di ${name_s}$"
        for elem in subs:
            if not is_seq_of_type(elem, "subs", subs_type)[0]:
                return submission_string + f"{evaluation_format('No', 0,pt_red,index_pt)}" + f"Attenzione la sottosequenza ${elem}$ non è del tipo richiesto."
        return submission_string + f"{evaluation_format('Ammissibile', pt_green,pt_red,index_pt)}"
    

Si consideri la seguente sequenza di numeri naturali:<br/><br/>[63, 18, 55, 81, 7, 9, 25, 13, 31, 47, 70, 83, 4, 32, 16, 61, 43, 20, 15, 54, 63, 99, 43, 14, 27]

__Richiesta 0 [5 punti]__: Fornire una sottosequenza $subs1$ <b>strettamente crescente</b> di $s$ che sia la più lunga possibile.

In [ ]:
#Inserisci la risposta (per esempio, [1,2,3])
subs1=[]

In [ ]:
display(Markdown(is_subseq_of_type(s, 's', subs1, 'subs1', 'SC', pt_green=1, pt_red=5,index_pt=0)))

__Richiesta 1 [5 punti]__: Fornire una sottosequenza $subs2$ di $s$, la più lunga possibile, che sia <b>strettamente decrescente</b> e non includa gli elementi di $s$ dalla posizione 5 alla posizione 8.

In [ ]:
#Inserisci la risposta (per esempio, [1,2,3])
subs2=[]

In [ ]:
display(Markdown(is_subseq_of_type(s, 's', subs2, 'subs2', 'SD', pt_green=1, pt_red=5,index_pt=1, start_banned_interval=5, end_banned_interval=8)))

__Richiesta 2 [10 punti]__: Fornire una sottosequenza $subs3$ di $s$, la più lunga possibile, che sia <b>strettamente crescente</b> e che includa l'elemento di $s$ in posizione 15

In [ ]:
#Inserisci la risposta (per esempio, [1,2,3])
subs3=[]

In [ ]:
display(Markdown(is_subseq_of_type(s, 's', subs3, 'subs3', 'SC', pt_green=1, pt_red=10,index_pt=2, forced_ele_pos=15)))

__Richiesta 3 [10 punti]__: Una sequenza è detta <b>una Z-sequenza</b> <it>(non-crescente con al più un ripensamento)</it>. Fornire una sottosequenza $subs4$ di $s$, la più lunga possibile, che sia di questo tipo.

In [ ]:
#Inserisci la risposta (per esempio, [1,2,3])
subs4=[]

In [ ]:
display(Markdown(is_subseq_of_type(s, 's', subs4, 'subs4', 'Z', pt_green=1, pt_red=10,index_pt=3)))

__Richiesta 4 [20 punti]__: Qual è il minor numero possibile di colori _C_ con cui si possa colorare gli elementi della sequenza $s$ in modo che, per ogni colore, la sottosequenza degli elementi di quel colore sia monotona <b>non-crescente</b>? Specificare il colore da assegnare a ogni elemento, usando come colori i numeri da 1 a _C_ (esempio: [1,2,3,4,2,1,3,1,1,...])

In [ ]:
#Inserisci la risposta (per esempio, [1,2,3])
subs5=[]

In [ ]:
display(Markdown(eval_coloring(s, 's', subs5, 'subs5', 'NC', pt_green=2, pt_red=20,index_pt=4)))

In [ ]:
import os
import time
from urllib.request import urlopen
import nbformat
from traitlets.config import Config
from nbconvert import HTMLExporter
from nbconvert import RSTExporter, NotebookExporter
from IPython.display import Image, HTML, Javascript
from datetime import datetime

import shutil
last ='preview_last/' #path of last submission
old = 'preview_old/' #path of prevous submissions

#hide alert buttons
def hide_w(w):
    for e in w:
        e.layout.visibility = 'hidden'

#show alert buttons
def show_w(w):
    for e in w:
        e.layout.visibility = 'visible'

def are_you_sure():
    show_w(alert_buttons)

label = widgets.Label(value="Un tuo elaborato era già stato precedentemente prodotto. Vuoi rimpiazzarlo con il presente?")
yes = widgets.Button(description="Si", tooltip="Produce il nuovo elaborato. Esso diviene la versione attuale prendendo il posto del precedente")
no = widgets.Button(description="No")

alert_buttons = [label,yes,no]
hide_w(alert_buttons)

#move all files from last to old and save the new preview
def yes_clicked(b):
    files = os.listdir(last)
    for f in files:
        if f == '.ipynb_checkpoints':
            continue
        if not os.path.isdir('./preview_old'):
            os.mkdir(old)
        shutil.move(last+f, old)
    display(Javascript('IPython.notebook.save_checkpoint();'))
    now = datetime.now() # current date and time
    date_time = now.strftime("%Y-%m-%d-%H-%M-%S")
    hname = 'Esercizio_03.ipynb'[:-6] + '_' + date_time
    ! jupyter nbconvert Esercizio_03.ipynb --to html_embed --output=$hname --output-dir=./preview_last/ --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags="['noexport']"
    display(Javascript('window.open("./preview_last/' + hname + '.html")'))
    hide_w(alert_buttons)
def no_clicked(b):
    hide_w(alert_buttons)


def generate_preview_HTML(_):
    if not os.path.isdir('./preview_last'):
        os.mkdir(last)
    directory= os.listdir(last)
    if len(directory) <1:
        display(Javascript('IPython.notebook.save_checkpoint();'))
        now = datetime.now() # current date and time
        date_time = now.strftime("%Y-%m-%d-%H-%M-%S")
        hname = 'Esercizio_03.ipynb'[:-6] + '_' + date_time
        ! jupyter nbconvert Esercizio_03.ipynb --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags="['noexport']" --to html_embed --output=$hname --output-dir=./preview_last/
        display(Javascript('window.open("./preview_last/' + hname + '.html")'))
    else:
        are_you_sure()


button = widgets.Button(description="Salva & Esporta", tooltip="Esporta il foglio Jupyter in HTML nella cartella preview")
output = widgets.Output()
button.on_click(generate_preview_HTML)
yes.on_click(yes_clicked)
no.on_click(no_clicked)
h_box1 = widgets.HBox([button])
h_box2 = widgets.HBox([label,yes,no])
display(widgets.VBox([h_box1,h_box2]))
